In [1]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1260
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [2]:
function kalman_filter_graph()
    model = Model(DefaultMessageGate())
    
    x_prev_add = add!(model, AdditionNode())
    
    x_prior = add!(model, datavar(:x_prior, Normal{Float64}))
    add_1   = add!(model, constvar(:add_1, 1.0))

    connect!(x_prev_add, :in1, x_prior, 1)
    connect!(x_prev_add, :in2, add_1, 1)
    
    noise = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    
    add_x_and_noise = add!(model, AdditionNode())
    
    x = add!(model, simplerandomvar(:x))
    
    connect!(x_prev_add, :out, x, 1)
    connect!(add_x_and_noise, :in1, x, 2)
    connect!(add_x_and_noise, :in2, noise, 1)
    
    y = add!(model, datavar(:y, Float64))
    
    connect!(add_x_and_noise, :out, y, 1)
    
    activate!(model, x_prev_add)
    activate!(model, add_x_and_noise)
    
    return x_prior, x, y
end

function kalman(data)
    N = length(data)
    
    x_prior, x, y = kalman_filter_graph()

    link(x_prior, getmarginal(x))
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman (generic function with 1 method)

In [3]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [4]:
@time kalman(data); # Initial compilation

  3.849366 seconds (11.71 M allocations: 605.599 MiB, 3.67% gc time)


In [5]:
@time kalman(data); # Subsequent runs

  0.001222 seconds (14.30 k allocations: 637.859 KiB)


In [7]:
@btime kalman($data) # Performance benchmark

  894.856 μs (14301 allocations: 637.86 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-11.582151154021464, σ=14.002800840280099)
 Normal{Float64}(μ=-0.38424997201726396, σ=9.950371902099892)
 Normal{Float64}(μ=9.062027220152988, σ=8.137884587711596)
 Normal{Float64}(μ=8.390722454936498, σ=7.053456158585984)
 Normal{Float64}(μ=8.095469893057302, σ=6.311944030978033)
 Normal{Float64}(μ=9.792777468998953, σ=5.763904177042351)
 Normal{Float64}(μ=7.055263269495753, σ=5.337605126836238)
 Normal{Float64}(μ=8.425286300596827, σ=4.993761694389224)
 Normal{Float64}(μ=10.19885505251716, σ=4.708816093480111)
 Normal{Float64}(μ=11.718844930918454, σ=4.467670516087703)
 Normal{Float64}(μ=11.867576100454759, σ=4.2601432284230505)
 Normal{Float64}(μ=11.346538784404709, σ=4.079085082240021)
 Normal{Float64}(μ=12.257694013626573, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=588.6102452884363, σ=0.582706653680032)
 Normal{Float64}(μ=589.5885821531751, σ=0.5822126418123303)
 Normal{Float64}(μ=590.6002115891946, σ=0.5817198842703781)
 N